<a href="https://colab.research.google.com/github/Syilun/ML20200716/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload() # upload kaggle.json colab的功能
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [2]:
!kaggle competitions download -c titanic

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 11.8MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 65.0MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.03MB/s]


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv', encoding='utf8')
test_df = pd.read_csv('test.csv', encoding='utf8')

In [5]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [8]:
datas = pd.concat([train_df, test_df], axis=0)

In [ ]:
datas = datas.drop(['PassengerId', 'Survived'],axis=1)

In [14]:
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [18]:
# Step1. 補缺失值
missing = datas.isna().sum()

# 篩選 series[[True...]]
missing[missing > 0]


Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [23]:
# 確認數字的類型: 1.連續 2.離散
# 補missing方法: 最常出現/不補
most = datas['Embarked'].value_counts().idxmax()
datas['Embarked'] = datas['Embarked'].fillna(most)

In [27]:
# 補數字類型
med = datas.median().drop('Pclass') # Pclass因為是離散，應該補最常出現的
d = datas.fillna(med)
d.isna().sum()

Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin       1014
Embarked       0
dtype: int64

In [28]:
d['Ticket'].value_counts()

CA. 2343         11
CA 2144           8
1601              8
347077            7
3101295           7
                 ..
28206             1
C.A. 42795        1
S.O./P.P. 251     1
350025            1
382653            1
Name: Ticket, Length: 929, dtype: int64

In [35]:
# 處理Name&Ticket

# def name(n):
#   n = n.split(',')[-1].split('.')[0]
#   return n.strip() # strip() 將左右兩邊的空白與換行改成換行
# datas['Name'].apply(name).value_counts()


Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Ms                2
Mlle              2
Major             2
Capt              1
Dona              1
Jonkheer          1
Mme               1
the Countess      1
Lady              1
Sir               1
Don               1
Name: Name, dtype: int64

In [39]:
# # 處理Name&Ticket

# def name(n):
#   n = n.split(',')[-1].split('.')[0]
#   return n.strip() # strip() 將左右兩邊的空白與換行改成換行
# v = datas['Name'].apply(name).value_counts()
# white = v.index[:4]
# white

Index(['Mr', 'Miss', 'Mrs', 'Master'], dtype='object')

In [ ]:
# # 處理Name&Ticket

# def name(n):
#   n = n.split(',')[-1].split('.')[0]
#   n = n.strip() # strip() 將左右兩邊的空白與換行改成換行
#   if n in white:
#     return n
#   else:
#     return None
# v = datas['Name'].apply(name).value_counts()
# white = v.index[:4]
# white

# # 缺失值不補可以用 One-hot encoding，把缺失值補為0,0,0也就是None

In [40]:
# 處理Name&Ticket

def name(n):
  n = n.split(',')[-1].split('.')[0]
  n = n.strip() # strip() 將左右兩邊的空白與換行改成換行
  if n in white:
    return n
  else:
    return None
datas['Name'] = datas['Name'].apply(name)
datas
# 缺失值不補可以用 One-hot encoding，把缺失值補為0,0,0也就是None

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,None,female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,Mr,male,NaN,0,0,359309,8.0500,NaN,S


In [43]:
v = datas['Ticket'].value_counts()
def ticket(t):
  global v
  return v[t]
datas['Ticket'].apply(ticket)
datas['Ticket'] = datas['Ticket'].apply(ticket)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,1,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,2,71.2833,C85,C
2,3,Miss,female,26.0,0,0,1,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,2,53.1000,C123,S
4,3,Mr,male,35.0,0,0,1,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,NaN,0,0,1,8.0500,NaN,S
414,1,None,female,39.0,0,0,3,108.9000,C105,C
415,3,Mr,male,38.5,0,0,1,7.2500,NaN,S
416,3,Mr,male,NaN,0,0,1,8.0500,NaN,S


In [ ]:
datas = datas.drop(['Cabin'], axis=1)
datas

In [50]:
datas = datas.reset_index(drop=True)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,Mr,male,22.0,1,0,1,7.2500,S
1,1,Mrs,female,38.0,1,0,2,71.2833,C
2,3,Miss,female,26.0,0,0,1,7.9250,S
3,1,Mrs,female,35.0,1,0,2,53.1000,S
4,3,Mr,male,35.0,0,0,1,8.0500,S
...,...,...,...,...,...,...,...,...,...
1304,3,Mr,male,NaN,0,0,1,8.0500,S
1305,1,None,female,39.0,0,0,3,108.9000,C
1306,3,Mr,male,38.5,0,0,1,7.2500,S
1307,3,Mr,male,NaN,0,0,1,8.0500,S


In [53]:
# One-hot encoding 處理類別字串
# 比如embarked中的 S C Q轉為3*3矩陣 100, 010, 001
# Pclass，1 2 3，有大小關係但不能以中位數來補
datas = pd.get_dummies(datas)

In [58]:
x_train = datas.iloc[0:len(train_df)]
x_predict = datas.iloc[len(train_df):]
y_train = train_df['Survived']


In [59]:
x_train

,Pclass,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,2,71.2833,0,0,0,1,1,0,1,0,0
2,3,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,1
3,1,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,1,13.0000,0,0,0,0,0,1,0,0,1
887,1,19.0,0,0,1,30.0000,0,1,0,0,1,0,0,0,1
888,3,NaN,1,2,4,23.4500,0,1,0,0,1,0,0,0,1
889,1,26.0,0,0,1,30.0000,0,0,1,0,0,1,1,0,0


In [ ]:
# iteratibeImputer() 看一欄位與其他欄位相關程度